# Часть первая - загрузка датасетов

### 1. Подключим все необходимые библиотеки и *установим* те, которых нет в среде **по умолчанию**.

In [1]:
!pip install kaggle
!pip install pyTelegramBotAPI

import os
import cv2
import numpy as np
import tqdm
import joblib
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from skimage.feature import hog
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

### 2. Подключимся к *базе данных* и загрузим в среду выполнения *датасеты* необходимые для обучения моделей.

In [2]:
!mkdir -p ~/.kaggle

!mv /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API key setup complete.")

mv: cannot stat '/content/kaggle.json': No such file or directory
Kaggle API key setup complete.


In [3]:
import kagglehub

path = kagglehub.dataset_download("ashishjangra27/face-mask-12k-images-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/ashishjangra27/face-mask-12k-images-dataset/versions/1


In [4]:
!rm -rf data
!rm -rf dataset

!mkdir -p dataset/FaceMask

!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

!unzip -q face-mask-12k-images-dataset.zip

!mv "Face Mask Dataset"/* dataset/FaceMask/

!rm face-mask-12k-images-dataset.zip
!rm -rf "Face Mask Dataset"

print("Dataset downloaded and extracted to dataset/FaceMask/")
print("Please re-run the main code cell (MQlGaZqbU7a6) now.")

Dataset URL: https://www.kaggle.com/datasets/ashishjangra27/face-mask-12k-images-dataset
License(s): CC0-1.0
 90% 297M/330M [00:00<00:00, 340MB/s]
100% 330M/330M [00:00<00:00, 369MB/s]
Dataset downloaded and extracted to dataset/FaceMask/
Please re-run the main code cell (MQlGaZqbU7a6) now.


### 3. Установим *пути*, куда были загружены *датасеты*.

In [5]:
!mkdir -p dataset/FaceMask/Train
!mkdir -p dataset/FaceMask/Validation
!mkdir -p dataset/FaceMask/Test

dataset_path = "dataset/FaceMask/"
train_dir = dataset_path + "Train/"
val_dir = dataset_path + "Validation/"
test_dir = dataset_path + "Test/"

### 4. *Генератор данных*

In [6]:
IMG_SIZE = 128
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


# Часть вторая - обучение моделей

### 1. Обучаем первую модель - *HOG + SVM*

In [7]:
def extract_hog_features_from_dir(directory):
    features = []
    labels = []

    for class_name in ["WithMask", "WithoutMask"]:
        class_dir = os.path.join(directory, class_name)
        label = 0 if class_name == "WithMask" else 1

        for fname in os.listdir(class_dir):
            path = os.path.join(class_dir, fname)

            img = cv2.imread(path)
            if img is None:
                continue

            img = cv2.resize(img, (64, 64))  # SVM uses fixed size 64x64
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            hog_vec = hog(
                gray,
                orientations=9,
                pixels_per_cell=(8, 8),
                cells_per_block=(2, 2),
                block_norm="L2-Hys"
            )

            features.append(hog_vec)
            labels.append(label)

    return np.array(features), np.array(labels)


print("Extracting HOG features...")
X_train_hog, y_train_hog = extract_hog_features_from_dir(train_dir)
X_val_hog,   y_val_hog   = extract_hog_features_from_dir(val_dir)

print("Training SVM...")
svm_model = make_pipeline(StandardScaler(), SVC(kernel="linear", probability=True))
svm_model.fit(X_train_hog, y_train_hog)

print("SVM Accuracy:")
pred = svm_model.predict(X_val_hog)
print(classification_report(y_val_hog, pred))

joblib.dump(svm_model, "svm_hog.pkl")
print("SVM model saved.")

Extracting HOG features...
Training SVM...
SVM Accuracy:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       400
           1       0.98      0.97      0.98       400

    accuracy                           0.98       800
   macro avg       0.98      0.98      0.98       800
weighted avg       0.98      0.98      0.98       800

SVM model saved.


### 2. Обучаем вторую модель - *SIMPLE CNN*

In [8]:
cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

cnn.compile(
    loss="binary_crossentropy",
    optimizer=Adam(1e-4),
    metrics=["accuracy"]
)

history_cnn = cnn.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

cnn.save("cnn_model.keras")
print("CNN saved.")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 335s 1s/step - accuracy: 0.8304 - loss: 0.3746 - val_accuracy: 0.9638 - val_loss: 0.1052
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 392s 1s/step - accuracy: 0.9500 - loss: 0.1357 - val_accuracy: 0.9812 - val_loss: 0.0607
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 340s 1s/step - accuracy: 0.9632 - loss: 0.1025 - val_accuracy: 0.9762 - val_loss: 0.0513
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 335s 1s/step - accuracy: 0.9728 - loss: 0.0801 - val_accuracy: 0.9912 - val_loss: 0.0309
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 336s 1s/step - accuracy: 0.9782 - loss: 0.0659 - val_accuracy: 0.9912 - val_loss: 0.0255
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 337s 1s/step - accuracy: 0.9765 - loss: 0.0627 - val_accuracy: 0.9900 - val_loss: 0.0229
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 331s 1s/step - accuracy: 0.9823 - loss: 0.0515 - val_accuracy: 0.9937 - val_loss: 0.0222
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 336s 1s/step - accuracy: 0.9760 - loss: 0.0657 - val_accu

### 3. Обучаем третью модель - *TRANSFER LEARNING ( "MobileNetV2" )*

In [9]:
base = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(128,128,3)
)

base.trainable = False

mobilenet = Sequential([
    base,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

mobilenet.compile(
    loss="binary_crossentropy",
    optimizer=Adam(1e-4),
    metrics=["accuracy"]
)

history_mob = mobilenet.fit(
    train_gen,
    validation_data=val_gen,
    epochs=7
)

mobilenet.save("mobilenet_model.keras")
print("MobileNetV2 saved.")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/7
313/313 ━━━━━━━━━━━━━━━━━━━━ 168s 516ms/step - accuracy: 0.9002 - loss: 0.2393 - val_accuracy: 0.9887 - val_loss: 0.0370
Epoch 2/7
313/313 ━━━━━━━━━━━━━━━━━━━━ 159s 506ms/step - accuracy: 0.9832 - loss: 0.0554 - val_accuracy: 0.9937 - val_loss: 0.0252
Epoch 3/7
313/313 ━━━━━━━━━━━━━━━━━━━━ 158s 505ms/step - accuracy: 0.9853 - loss: 0.0442 - val_accuracy: 0.9937 - val_loss: 0.0184
Epoch 4/7
313/313 ━━━━━━━━━━━━━━━━━━━━ 157s 503ms/step - accuracy: 0.9865 - loss: 0.0355 - val_accuracy: 0.9937 - val_loss: 0.0173
Epoch 5/7
313/313 ━━━━━━━━━━━━━━━━━━━━ 157s 501ms/step - accuracy: 0.9888 - loss: 0.0318 - val_accuracy: 0.9975 - val_loss: 0.0109
Epoch 6/7
313/313 ━━━━━━━━━━━━━━━━━━━━ 158s 506ms/step - accuracy: 0.9863 - loss: 0.0398 - val_accuracy: 0.9950 - val_loss: 0.0132
Epoch 7/7
313/313 ━━━━━━━━━━━━━━━━━━━━ 160s 510ms/step - accuracy: 0.9901 - loss: 0.0241 - val_accuracy: 0.9950 - val_loss: 0.0150
MobileNetV2 saved.


# Часть третья - Telegram Bot

### 1. Подключим необходимые *библиотеки*

In [38]:
import telebot
import numpy as np
import cv2
from skimage.feature import hog
from keras.models import load_model
import joblib
from io import BytesIO

### 2. Подгрузим *обученные ранее модели*

In [39]:
svm_model = joblib.load("svm_hog.pkl")
cnn_model = load_model("cnn_model.keras")
mobilenet_model = load_model("mobilenet_model.keras")

print("All models loaded.")

All models loaded.


### 3. Напишем функцию - *подготовка фотографии*

In [40]:
def preprocess_for_hog(img):
    img = cv2.resize(img, (64, 64))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    hog_vec = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm="L2-Hys"
    )

    return hog_vec.reshape(1, -1)

def preprocess_for_cnn(img):
    img = cv2.resize(img, (128, 128))
    img = img.astype("float32") / 255.0
    return np.expand_dims(img, axis=0)


def preprocess_for_mobilenet(img):
    img = cv2.resize(img, (128, 128))
    img = img.astype("float32") / 255.0
    return np.expand_dims(img, axis=0)

### 4. Напишем функцию - *обнаружение маски*

In [41]:
def predict_mask(img):
    hog_feat = preprocess_for_hog(img.copy())
    svm_pred = svm_model.predict(hog_feat)[0]
    svm_label = "Mask" if svm_pred == 0 else "No Mask"

    cnn_img = preprocess_for_cnn(img.copy())
    cnn_prob = cnn_model.predict(cnn_img)[0][0]
    cnn_label = "Mask" if cnn_prob < 0.5 else "No Mask"

    mob_img = preprocess_for_mobilenet(img.copy())
    mob_prob = mobilenet_model.predict(mob_img)[0][0]
    mob_label = "Mask" if mob_prob < 0.5 else "No Mask"

    return svm_label, cnn_label, mob_label

### 5. *Начало работы*

In [ ]:
bot = telebot.TeleBot("MyToken")

@bot.message_handler(commands=['start'])
def start(msg):
    bot.send_message(
        msg.chat.id,
        "👋 Привет! Отправь мне фото лица, и я скажу — человек в маске или нет."
    )

### 6. Обработка случая - бот получил *фото*

In [43]:
@bot.message_handler(content_types=['photo'])
def handle_photo(msg):

    file_id = msg.photo[-1].file_id
    file_info = bot.get_file(file_id)
    img_bytes = bot.download_file(file_info.file_path)

    img_array = np.frombuffer(img_bytes, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    if img is None:
        bot.send_message(msg.chat.id, "❌ Ошибка: не удалось обработать изображение.")
        return

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    svm_label, cnn_label, mob_label = predict_mask(img)

    text = (
        "🔍 *Результаты анализа:*\n\n"
        f"🟦 *SVM (HOG):* `{svm_label}`\n"
        f"🟩 *CNN:* `{cnn_label}`\n"
        f"🟧 *MobileNet:* `{mob_label}`\n"
    )

    bot.send_message(msg.chat.id, text, parse_mode="Markdown")

### 7. Обработка случая - бот получил *файл*

In [44]:
@bot.message_handler(content_types=['document'])
def handle_document(msg):

    mime = msg.document.mime_type
    if not mime.startswith("image/"):
        bot.send_message(msg.chat.id, "❌ Пожалуйста, отправьте изображение (jpg/png).")
        return

    file_id = msg.document.file_id
    file_info = bot.get_file(file_id)
    img_bytes = bot.download_file(file_info.file_path)

    img_array = np.frombuffer(img_bytes, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    if img is None:
        bot.send_message(msg.chat.id, "❌ Ошибка: не удалось обработать изображение.")
        return

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    svm_label, cnn_label, mob_label = predict_mask(img)

    text = (
        "🔍 *Результаты анализа:*\n\n"
        f"🟦 *SVM (HOG):* `{svm_label}`\n"
        f"🟩 *CNN:* `{cnn_label}`\n"
        f"🟧 *MobileNet:* `{mob_label}`\n"
    )

    bot.send_message(msg.chat.id, text, parse_mode="Markdown")

### 8. *Включение* бота и *оповещение*

In [45]:
print("Bot is running...")
bot.polling(none_stop=True)

Bot is running...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
